In [3]:
##rf第一步：数据读取

import pandas as pd
file_path = "C:/Users/HP/Desktop/final/train.csv"

# 尝试不同编码格式
try:
    data = pd.read_csv(file_path, encoding="utf-8")  # UTF-8 编码
except UnicodeDecodeError:
    try:
        data = pd.read_csv(file_path, encoding="gbk")  # GBK 编码（适用于中文）
    except UnicodeDecodeError:
        data = pd.read_csv(file_path, encoding="latin1")  # Latin-1 编码（适用于部分 ANSI 文件）
print("数据读取成功！")

# 交易时间和建筑面积的数据提取
#data['交易时间'] = pd.to_datetime(data['交易时间'], format='%Y/%m/%d')  # 转换为日期格式
#data['交易时间'] = data['交易时间'].dt.year  # 提取年份
# 去掉 "㎡" 并转换为 float 类型
data['建筑面积'] = data['建筑面积'].str.replace('㎡', '', regex=False).astype(float)



数据读取成功！


In [5]:
#第二步：数据处理
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder

# 1：选取特征和目标变量
df = data.copy()  # 假设你已经完成前面的异常值处理等
df['建筑面积'] = pd.to_numeric(df['建筑面积'], errors='coerce')
df['单价'] = df['价格'] / df['建筑面积']
df['log_单价'] = np.log(df['单价'])
df['log_面积'] = np.log(df['建筑面积'] + 1)

# 2：处理分类变量（独热编码）
categorical_cols = ['城市', '板块', '区域', '环线', '房屋户型', '所在楼层', '房屋朝向', '装修情况', 'log_面积', '别墅类型', '房屋用途', '房屋年限',   
        '交通出行','年份']
categorical_cols_to_encode = ['年份','城市', '板块','环线', '房屋户型', '所在楼层', '房屋朝向', '装修情况','别墅类型', '房屋用途', '房屋年限','交通出行' ]
df = pd.get_dummies(df, columns=categorical_cols_to_encode, drop_first=True)
print(df.head())

   区域        价格    建筑面积  配备电梯        交易时间  交易权属  产权所属  周边配套             单价  \
0  79   6564200   52.30     0   2020/8/11     1     0     1  125510.516252   
1  43   4174000  127.44     0   2020/3/13     1     0     1   32752.667922   
2  97  16310000  228.54     0  2020/11/21     1     0     0   71366.062834   
3  62   2834600   43.60     1   2019/2/14     1     0     0   65013.761468   
4  62   1954000   39.85     1   2019/4/26     1     0     0   49033.877039   

      log_单价  ...  装修情况_2  装修情况_3  别墅类型_1  别墅类型_2  别墅类型_3  房屋用途_1  房屋年限_1  \
0  11.740145  ...   False    True   False   False   False    True   False   
1  10.396740  ...   False    True   False   False   False    True   False   
2  11.175578  ...   False    True   False   False   False    True   False   
3  11.082354  ...   False    True   False   False   False   False   False   
4  10.800267  ...   False    True   False   False   False   False   False   

   房屋年限_2  房屋年限_5  交通出行_1  
0   False    True   False  
1   False   

In [ ]:
#异常值处理

import numpy as np

# 计算建筑面积的 IQR 并去除异常值
Q1_area = data['建筑面积'].quantile(0.25)
Q3_area = data['建筑面积'].quantile(0.75)
IQR_area = Q3_area - Q1_area
lower_bound_area = Q1_area - 1.5 * IQR_area
upper_bound_area = Q3_area + 1.5 * IQR_area

# 计算房价（单位面积价格）的 IQR 并去除异常值
Q1_price = data['价格'].quantile(0.25)
Q3_price = data['价格'].quantile(0.75)
IQR_price = Q3_price - Q1_price
lower_bound_price = Q1_price - 1.5 * IQR_price
upper_bound_price = Q3_price + 1.5 * IQR_price

# 过滤数据
filtered_data = data[
    (data['建筑面积'] >= lower_bound_area) & (data['建筑面积'] <= upper_bound_area) & 
    (data['价格'] >= lower_bound_price) & (data['价格'] <= upper_bound_price)
]

In [7]:
# 第三步：训练集划分
#1：设定特征X和目标y
feature_cols = [col for col in df.columns if col.startswith('log_面积') or 
                col.startswith(tuple(categorical_cols)) or 
                col.startswith(tuple([c + "_" for c in categorical_cols]))]
X = df[feature_cols]
y = df['log_单价']
y_region_mean = y.mean()


# 2：训练测试集划分
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#3：目标编码
# 计算训练集中的区域均值
region_mean_map = X_train.join(y_train).groupby('区域')['log_单价'].mean()
# 用训练集均值填充训练集
# 区域目标编码结果单独保存
#X_train_region_encoded = X_train['区域'].map(region_mean_map)
# 把它添加到 X_train，再执行这一行
#X_train['区域_encoded'] = X_train_region_encoded
X_train['区域_encoded']=X_train['区域'].map(region_mean_map)

# 在测试集上进行编码时，只使用训练集信息
X_test['区域_encoded'] = X_test['区域'].map(region_mean_map)

print(X_train.head())
print(y_region_mean)

       区域    log_面积  年份_2016  年份_2017  年份_2018  年份_2019  年份_2020  年份_2021  \
35981  68  4.753590    False    False    False    False    False     True   
19903  67  4.939426    False    False    False    False    False     True   
79323  93  4.254193    False    False    False    False    False    False   
35652  65  4.509650    False    False    False    False    False     True   
539    62  3.797734    False    False    False    False     True    False   

       年份_2022   城市_1  ...  装修情况_3  别墅类型_1  别墅类型_2  别墅类型_3  房屋用途_1  房屋年限_1  \
35981    False  False  ...    True   False   False   False    True   False   
19903    False   True  ...    True   False   False   False    True   False   
79323     True  False  ...   False   False   False   False    True   False   
35652    False  False  ...    True   False   False   False    True   False   
539      False  False  ...    True   False   False   False   False   False   

       房屋年限_2  房屋年限_5  交通出行_1  区域_encoded  
35981   False   False   

In [148]:
#第四步：随机森林参数选择
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 300, 500],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt']
}

rf = RandomForestRegressor(random_state=42)
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

print("最佳参数组合：", grid_search.best_params_)


Fitting 3 folds for each of 162 candidates, totalling 486 fits


D:\tools\anaconda\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
243 fits failed out of a total of 486.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
125 fits failed with the following error:
Traceback (most recent call last):
  File "D:\tools\anaconda\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "D:\tools\anaconda\Lib\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
  File "D:\tools\anaconda\Lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "D:\tools\anaconda\Lib\site-packages\sklearn\utils\_param_validation.

最佳参数组合： {'max_depth': 30, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}


In [321]:
# 第五步：训练随机森林模型
rf = RandomForestRegressor(n_estimators=100, max_depth=30, random_state=42)
rf.fit(X_train, y_train)

# 预测与评估
y_pred = rf.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"模型评估结果：RMSE = {rmse:.4f}, R² = {r2:.4f}")


模型评估结果：RMSE = 0.1569, R² = 0.9583


In [325]:
# 第六步：输出特征重要性
feature_importance = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': rf.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("特征重要性前10名：")
print(feature_importance.head(10))

特征重要性前10名：
        Feature  Importance
703  区域_encoded    0.858781
1        log_面积    0.034364
698      房屋用途_1    0.011147
677        环线_4    0.003927
676        环线_3    0.003335
0            区域    0.002837
675        环线_2    0.002824
693      装修情况_2    0.002793
213      板块_247    0.002365
702      交通出行_1    0.001916


In [327]:
#第七步.总价模型评估
# 1：还原预测值和实际值（从 log_单价 -> 单价 -> 总价）
y_pred_price = np.exp(y_pred)                      # 还原为单价
y_test_price = np.exp(y_test)                      # 实际单价

area_test = np.exp(X_test['log_面积']) - 1          # 还原建筑面积（对应 +1 的变换）

y_pred_total_price = y_pred_price * area_test
y_test_total_price = y_test_price * area_test

# 2：计算总价的 RMSE 和 R²
rmse_total = np.sqrt(mean_squared_error(y_test_total_price, y_pred_total_price))
r2_total = r2_score(y_test_total_price, y_pred_total_price)

print(f"总价预测结果：RMSE = {rmse_total:.2f}, R² = {r2_total:.4f}")


总价预测结果：RMSE = 1202400.78, R² = 0.8107


In [274]:
#6折交叉验证
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# —— 1. 准备 X_full、y_full —— 
feature_cols = [
    col for col in df.columns
    if col.startswith('log_面积')
    or col.startswith(tuple(categorical_cols))
    or col.startswith(tuple([c + "_" for c in categorical_cols]))
]
X_full = df[feature_cols].copy()    # 这里应保证 X_full 中包含了 '区域' 列
y_full = df['log_单价'].copy()

# 计算整体 y 的均值，以便给那些在训练折中未出现的“区域”赋值
global_y_mean = y_full.mean()

# —— 2. 定义 6 折交叉验证 —— 
kf = KFold(n_splits=6, shuffle=True, random_state=42)

# 用来保存每一折的总价 RMSE 和 R²
rmse_list = []
r2_list = []

# —— 3. 逐折训练与评估 —— 
for fold_idx, (train_idx, val_idx) in enumerate(kf.split(X_full), start=1):
    # 3.1 划分本折的训练集和验证集
    X_train_fold = X_full.iloc[train_idx].copy()
    X_val_fold   = X_full.iloc[val_idx].copy()
    y_train_fold = y_full.iloc[train_idx].copy()
    y_val_fold   = y_full.iloc[val_idx].copy()
    
    # 3.2 在本折训练集上计算“区域”目标编码映射
    #     groupby 后得到 Series：index 是区域名称，values 是该区域在本折训练集上的 log_单价 平均值
    region_mean_map = (
        X_train_fold[['区域']]
        .join(y_train_fold)
        .groupby('区域')['log_单价']
        .mean()
    )
    
    # 3.3 把本折训练集和验证集中 '区域' 映射成 '区域_encoded'
    X_train_fold['区域_encoded'] = X_train_fold['区域'].map(region_mean_map)
    X_val_fold  ['区域_encoded'] = X_val_fold ['区域'].map(region_mean_map)
    
    # 对于验证集中出现训练集中没有的“区域”，用训练集 y 的整体平均值来填充
    X_train_fold['区域_encoded'] = X_train_fold['区域_encoded'].fillna(y_train_fold.mean())
    X_val_fold  ['区域_encoded'] = X_val_fold ['区域_encoded'].fillna(y_train_fold.mean())
    
    # 3.4 如果 feature_cols 中已经包含了 '区域_encoded'，则无须再次添加到列表中
    #     否则，需要在 feature_cols_fold 中加入 '区域_encoded'
    feature_cols_fold = list(feature_cols)
    if '区域_encoded' not in feature_cols_fold:
        feature_cols_fold.append('区域_encoded')
    
    # 3.5 训练随机森林模型
    rf_cv = RandomForestRegressor(
        n_estimators=100, 
        max_depth=30, 
        random_state=42
    )
    rf_cv.fit(
        X_train_fold[feature_cols_fold],
        y_train_fold
    )
    
    # 3.6 在验证集上预测 log_单价
    y_val_pred_log = rf_cv.predict(X_val_fold[feature_cols_fold])
    
    # —— 4. 计算总价的 RMSE 与 R² —— 
    # 4.1 先把 log_单价 → 单价，然后再计算“总价”：单价 * 面积
    y_val_pred_price = np.exp(y_val_pred_log)      # 还原为 “单价”
    y_val_price      = np.exp(y_val_fold)          # 验证集真实 “单价”
    
    # 建筑面积也是 log 过 +1 的：log_面积 = np.log(面积 + 1)
    # 所以还原面积：area = exp(log_面积) - 1
    area_val = np.exp(X_val_fold['log_面积']) - 1
    
    y_val_pred_total = y_val_pred_price * area_val
    y_val_total      = y_val_price      * area_val
    
    # 4.2 计算本折的 RMSE 和 R²
    rmse_fold = np.sqrt(mean_squared_error(y_val_total, y_val_pred_total))
    r2_fold   = r2_score(y_val_total, y_val_pred_total)
    
    print(f"折数 {fold_idx} —— 总价预测 RMSE = {rmse_fold:.2f}, R² = {r2_fold:.4f}")
    
    rmse_list.append(rmse_fold)
    r2_list.append(r2_fold)

# —— 5. 输出交叉验证整体结果 —— 
mean_rmse = np.mean(rmse_list)
std_rmse  = np.std(rmse_list, ddof=1)
mean_r2   = np.mean(r2_list)
std_r2    = np.std(r2_list, ddof=1)

print("=" * 50)
print(f"6 折交叉验证 总价预测 RMSE 平均 = {mean_rmse:.2f}，标准差 = {std_rmse:.2f}")
print(f"6 折交叉验证 总价预测 R² 平均 = {mean_r2:.4f}，标准差 = {std_r2:.4f}")


折数 1 —— 总价预测 RMSE = 659352.52, R² = 0.9445
折数 2 —— 总价预测 RMSE = 986578.66, R² = 0.8532
折数 3 —— 总价预测 RMSE = 616097.93, R² = 0.9417
折数 4 —— 总价预测 RMSE = 567175.23, R² = 0.9509
折数 5 —— 总价预测 RMSE = 514305.96, R² = 0.9599
折数 6 —— 总价预测 RMSE = 606308.57, R² = 0.9505
6 折交叉验证 总价预测 RMSE 平均 = 658303.14，标准差 = 168101.23
6 折交叉验证 总价预测 R² 平均 = 0.9334，标准差 = 0.0398


In [331]:
##预测：1.数据处理：
#第八步：读取预测数据
test_path = "C:/Users/HP/Desktop/final/test_.xlsx"
test_df = pd.read_excel(test_path)

# 清洗建筑面积字段
test_df['建筑面积'] = test_df['建筑面积'].astype(str).str.replace('㎡', '', regex=False).astype(float)
# 记录原始建筑面积用于反还原价格
test_df['原始面积'] = test_df['建筑面积']

# 构造 log_面积（与训练集一致）
test_df['log_面积'] = np.log(test_df['建筑面积'] + 1)

# 保留训练中使用的所有特征列
test_categorical_cols = [
    '城市', '板块', '区域', '环线', '房屋户型', '所在楼层',
    '房屋朝向', '装修情况', 'log_面积', '别墅类型',
    '房屋用途', '房屋年限', '交通出行','年份'
]
test_df = test_df[test_categorical_cols]

# 与训练集相同的独热编码
test_df = pd.get_dummies(test_df, columns=['年份','城市', '板块','环线', '房屋户型', '所在楼层', '房屋朝向', '装修情况','别墅类型', '房屋用途', '房屋年限','交通出行'], drop_first=True)

# 区域目标编码
#test_df['区域_encoded'] = X_train_region_encoded
# 区域目标编码（用训练集映射）
#test_df['区域_encoded'] = X_train['区域'].map(region_mean_map)
test_df['区域_encoded'] = test_df['区域'].map(region_mean_map)
# 替换 NaN 为 y_region_mean
#test_df['区域_encoded'] = X_train['区域_encoded'].fillna(y_region_mean)
test_df['区域_encoded'] = test_df['区域_encoded'].fillna(y_region_mean)

# 补全缺失列
for col in X.columns:
    if col not in test_df.columns:
        test_df[col] = 0

# 多余列删除（如果测试集编码生成了训练集中没有的列）
#test_df = test_df[X.columns]  # 只保留训练集用到的列

# 确保列顺序一致
test_df = test_df[X_train.columns]

print(test_df.head())


   区域    log_面积  年份_2016  年份_2017  年份_2018  年份_2019  年份_2020  年份_2021  \
0  45  5.348059        0        0        0        0        0        0   
1  45  5.104065        0        0        0        0        0        0   
2  43  4.643621        0        0        0        0        0        0   
3  39  4.706462        0        0        0        0        0        0   
4  79  4.063885        0        0        0        0        0        0   

   年份_2022   城市_1  ...  装修情况_3  别墅类型_1  别墅类型_2  别墅类型_3  房屋用途_1  房屋年限_1  \
0        0  False  ...    True   False   False   False    True   False   
1        0  False  ...    True   False   False   False    True   False   
2        0  False  ...   False   False   False   False    True   False   
3        0  False  ...   False   False   False   False    True   False   
4        0  False  ...   False   False   False   False    True   False   

   房屋年限_2  房屋年限_5  交通出行_1  区域_encoded  
0    True   False   False   10.846593  
1   False    True    True   10.84659

In [333]:
#2. 进行预测（预测log_单价）
test_rf_log_unit_price = rf.predict(test_df)

# 反变换得到总价
test_price_rf = np.exp(test_rf_log_unit_price) * test_df['log_面积'].apply(lambda x: np.exp(x) - 1)

# 准备保存的结果
result_df_rf = pd.DataFrame({
    "ID": range(len(test_price_rf)),
    "Price": test_price_rf  # 保留原始浮点数
})

# 保存结果为CSV
output_path = "C:/Users/HP/Desktop/final/prediction_result_rf_3.csv"
result_df_rf.to_csv(output_path, index=False)
print(f"预测结果已保存至: {output_path}")



预测结果已保存至: C:/Users/HP/Desktop/final/prediction_result_rf_3.csv


In [ ]:
##xgboost：1.参数选取
import optuna
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# 假设已有 X, y
# X, y = ...

# 拆分训练/测试集（注意命名不能混乱）
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
def objective(trial):
    param = {
        'verbosity': 0,
        'objective': 'reg:squarederror',
        'booster': 'gbtree',
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0),
        'random_state': 42  # 保4证每次结果一致
    }

    model = xgb.XGBRegressor(**param)
    
    model.fit(
        X_train, y_train,
        eval_set=[(X_test, y_test)],
        #early_stopping_rounds=30,
        verbose=False
    )

    y_pred = model.predict(X_test)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    return rmse

# 创建 Study 对象
study = optuna.create_study(direction='minimize')

# 启动搜索
study.optimize(objective, n_trials=50, timeout=600)  # 运行最多5分钟或20轮

# 打印结果
print("Best trial:")
print(study.best_trial.params)


In [ ]:
best_params = study.best_trial.params
model = xgb.XGBRegressor(**best_params)
model.fit(X_train, y_train)  # 用训练集数据重新训得到模型评估


In [ ]:
#2.模型评估
y_pred = model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print(f"XGBoost模型评估结果：RMSE = {rmse:.4f}, R² = {r2:.4f}")



In [ ]:
#3.特征重要性
import pandas as pd

# 获取特征重要性（使用 gain 作为度量）
importance_dict = model.get_booster().get_score(importance_type='gain')

# 转换为 DataFrame 并排序
importance_df = pd.DataFrame({
    'Feature': list(importance_dict.keys()),
    'Gain': list(importance_dict.values())
}).sort_values(by='Gain', ascending=False)

# 输出结果
print("\n📊 特征重要性（按 Gain 排序）：")
print(importance_df.to_string(index=False))


In [ ]:
#4.总价模型评估
# 第一步：还原预测值和实际值（从 log_单价 -> 单价 -> 总价）
y_pred_price = np.exp(y_pred)                      # 还原为单价
y_test_price = np.exp(y_test)                      # 实际单价

area_test = np.exp(X_test['log_面积']) - 1          # 还原建筑面积（对应 +1 的变换）

y_pred_total_price = y_pred_price * area_test
y_test_total_price = y_test_price * area_test

# 第二步：计算总价的 RMSE 和 R²
rmse_total = np.sqrt(mean_squared_error(y_test_total_price, y_pred_total_price))
r2_total = r2_score(y_test_total_price, y_pred_total_price)

print(f"总价预测结果：RMSE = {rmse_total:.2f}, R² = {r2_total:.4f}")


In [ ]:
##预测：1.数据处理：
#第八步：读取预测数据
test_path = "C:/Users/HP/Desktop/final/test_.xlsx"
test_df = pd.read_excel(test_path)

# 清洗建筑面积字段
test_df['建筑面积'] = test_df['建筑面积'].astype(str).str.replace('㎡', '', regex=False).astype(float)
# 记录原始建筑面积用于反还原价格
test_df['原始面积'] = test_df['建筑面积']

# 构造 log_面积（与训练集一致）
test_df['log_面积'] = np.log(test_df['建筑面积'] + 1)

# 保留训练中使用的所有特征列
test_categorical_cols = [
    '城市', '板块', '区域', '环线', '房屋户型', '所在楼层',
    '房屋朝向', '装修情况', 'log_面积', '别墅类型',
    '房屋用途', '房屋年限', '交通出行','年份'
]
test_df = test_df[test_categorical_cols]

# 与训练集相同的独热编码
test_df = pd.get_dummies(test_df, columns=['城市', '板块'], drop_first=True)

# 区域目标编码
#test_df['区域_encoded'] = X_train_region_encoded
# 区域目标编码（用训练集映射）
#test_df['区域_encoded'] = X_train['区域'].map(region_mean_map)
test_df['区域_encoded'] = test_df['区域'].map(region_mean_map)
# 替换 NaN 为 y_region_mean
#test_df['区域_encoded'] = X_train['区域_encoded'].fillna(y_region_mean)
test_df['区域_encoded'] = test_df['区域_encoded'].fillna(y_region_mean)

# 补全缺失列
for col in X.columns:
    if col not in test_df.columns:
        test_df[col] = 0

# 多余列删除（如果测试集编码生成了训练集中没有的列）
#test_df = test_df[X.columns]  # 只保留训练集用到的列

# 确保列顺序一致
test_df = test_df[X_train.columns]

print(test_df.head())


In [ ]:
#2. 进行预测（预测log_单价）
test_xgboost_log_unit_price = model.predict(test_df)

# 反变换得到总价
test_price_xgboost = np.exp(test_xgboost_log_unit_price) * test_df['log_面积'].apply(lambda x: np.exp(x) - 1)

# 准备保存的结果
result_df_xgboost = pd.DataFrame({
    "ID": range(len(test_price_xgboost)),
    "Price": test_price_xgboost  # 保留原始浮点数
})

# 保存结果为CSV
output_path = "C:/Users/HP/Desktop/final/prediction_result_xg_3.csv"
result_df_xgboost.to_csv(output_path, index=False)
print(f"预测结果已保存至: {output_path}")


In [ ]:
from tensorflow.keras.layers import Input, Embedding, Flatten, Concatenate, Dense
from tensorflow.keras.models import Model

# --------- 1. 数值潜在特征输入 ---------
num_latent_input = Input(shape=(encoding_dim,), name="numeric_latent")

# --------- 2. 板块 Embedding 输入 ---------
#  假设 le_block.classes_ 数量为 num_blocks 个（即板块类别数）
num_blocks = len(le_block.classes_)

block_input = Input(shape=(1,), name="block_id")  # 传入一个整数 ID
# Embedding 层将板块 ID（0..num_blocks-1）映射到 4 维向量
block_embedding = Embedding(
    input_dim=num_blocks,
    output_dim=4,          # 4 维嵌入，也可调
    #input_length=1,
    name="block_embedding"
)(block_input)
block_vec = Flatten(name="block_flatten")(block_embedding)  # 展平成 (None, 4)

# --------- 3. 所有 one-hot 输入 ---------
cat_input = Input(shape=(X_cat_train.shape[1],), name="categorical_onehot")


# --------- 4. 拼接所有特征 ---------
concat = Concatenate(name="concat_all")(
    [num_latent_input, block_vec, cat_input]
)



# --------- 5. 全连接层做回归 ---------
x = Dense(32, activation='relu', name="dense_1")(concat)
x = Dense(16, activation='relu', name="dense_2")(x)
output = Dense(1, activation='linear', name="price_output")(x)

# --------- 6. 定义模型 ---------
model = Model(
    inputs=[num_latent_input, block_input, cat_input],
    outputs=output
)

# 打印模型结构，检查无误
model.summary()


In [ ]:
model.compile(
    optimizer='adam',
    loss='mse',            # 均方误差
    metrics=['mae']        # 平均绝对误差
)

# 准备训练时要传给 model.fit 的输入格式
train_inputs = {
    "numeric_latent": X_num_train_latent,      # Autoencoder Latent (n_train, 2)
    "block_id": X_block_train,                 # 板块 ID (n_train,)
    "categorical_onehot": X_cat_train          # 所有 one-hot 编码特征 (n_train,  独热维度)
}
test_inputs = {
    "numeric_latent": X_num_test_latent,
    "block_id": X_block_test,
    "categorical_onehot": X_cat_test
}

# 训练
history = model.fit(
    train_inputs,
    y_train,
    validation_split=0.1,  # 留一部分做验证
    epochs=50,
    batch_size=32,
    verbose=1
)


In [ ]:
# 在测试集上预测
y_pred = model.predict(test_inputs)

# 如果你的 y_test 是“log_单价”，可以计算 RMSE、R2
from sklearn.metrics import mean_squared_error, r2_score
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2  = r2_score(y_test, y_pred)
print(f"测试集 RMSE = {rmse:.4f}, R² = {r2:.4f}")


In [ ]:
import pandas as pd
import numpy as np

# ---------- 读取测试集并完成与训练时相同的数据清洗 ----------

test_path = "C:/Users/HP/Desktop/final/test_.xlsx"
df_test = pd.read_excel(test_path)

# （1）去掉“㎡”并转换成 float
df_test['建筑面积'] = (
    df_test['建筑面积']
    .astype(str)
    .str.replace('㎡', '', regex=False)
    .astype(float)
)

# （2）构造 log_面积（与训练逻辑一致）
df_test['log_面积'] = np.log(df_test['建筑面积'] + 1)

# （3）其它数值特征：一定要和训练时用的 numeric_cols 一致
numeric_cols = ['log_面积', '环线', '房屋户型', '所在楼层']
X_num_test = df_test[numeric_cols].values  # 形状：(n_test, 4)

# （4）板块 ID：必须用 训练阶段 的 le_block 去 transform
#     （如果某些板块在测试集里没出现，transform 会报错 —— 确保 le_block 是训练时 fit 后的那个）
# 先查看一下训练时 le_block 学到的类别列表
seen_blocks = set(le_block.classes_)
# 先提取原始板块标签
raw_blocks = df_test['板块'].astype(str).values

# （2）把测试集中训练时没见过的标签，先替换成一个“未知”占位符
# 这里把所有 unseen-label 都设为 'UNKNOWN'
df_test['板块_clean'] = df_test['板块'].astype(str).map(
    lambda x: x if x in seen_blocks else 'UNKNOWN'
)

# （3）在 LabelEncoder 里临时把 'UNKNOWN' 也当成一类：  
#     方法 A：直接在 transform 前把 unseen 映射到一个已有类别，比如第 0 个
#     方法 B：或者给 le_block.classes_ 里新增 'UNKNOWN'（但需要重新 fit）
#
# 我们用 方法 A，把所有 unseen 直接映射到已知的序号 0
# 即使 0 代表的原来是某个实际板块，但这样较简单

# 先拿到训练时 le_block 给每个标签的编码（字典形式）
mapping = {lbl: idx for idx, lbl in enumerate(le_block.classes_)}

# 定义一个函数：如果标签没在 mapping 里，就返回 0
def safe_encode(x):
    return mapping[x] if x in mapping else 0

# 把测试集“板块_clean”映射到整数
df_test['板块_id'] = df_test['板块_clean'].map(safe_encode).astype(int)

# 这样 df_test['板块_id'] 中所有原本 unseen 的都会是 0
X_block_test = df_test['板块_id'].values  # (n_test,)

# （5）one-hot 编码列：必须在训练时用到的 onehot_cols 列名，在测试集中一一提取
#     对不存在的列补 0
X_cat_test_list = []
for col in onehot_cols:  # onehot_cols 需从训练脚本中传递过来
    if col in df_test.columns:
        arr = df_test[col].values.reshape(-1, 1)  # 形状：(n_test,1)
    else:
        arr = np.zeros((df_test.shape[0], 1), dtype=float)
    X_cat_test_list.append(arr)

# 把所有 one-hot 列水平拼接成 ndarray
X_cat_test = np.hstack(X_cat_test_list)  # 形状：(n_test, len(onehot_cols))


In [ ]:
# 训练阶段 fit 好的 scaler、encoder 都要在此时直接复用
# scaler = StandardScaler().fit(X_num_train)  
# encoder = Model(inputs=num_input, outputs=encoded)

# (1) 标准化
X_num_test_scaled = scaler.transform(X_num_test)  # shape = (n_test, 4)

# (2) 用训练好的 encoder 提取 latent
X_num_test_latent = encoder.predict(X_num_test_scaled)  # shape = (n_test, encoding_dim)


In [ ]:
# 构造与训练时同名的输入字典
test_inputs = {
    "numeric_latent": X_num_test_latent,   # (n_test, encoding_dim)
    "block_id": X_block_test,              # (n_test,)
    "categorical_onehot": X_cat_test       # (n_test, len(onehot_cols))
}

# 打印检查一下三路输入的类型和形状
for key, val in test_inputs.items():
    print(f"{key}: type={type(val)}, shape={getattr(val, 'shape', '无shape')}")

# 如果都是 (n_test, …) 且 n_test 相同，就可以安全调用 predict
y_test_pred_log = model.predict(test_inputs)  # 形状：(n_test, 1)


In [ ]:

# 总价 = 单价 * 建筑面积 = exp(log_单价) * 建筑面积
df_test['预测_总价'] = np.exp(y_test_pred_log.flatten()) * df_test['建筑面积']

# 如果你只需要 log_单价 预测结果：
#df_test['预测_log_单价'] = y_test_pred_log

# 最终保存到 CSV 或 Excel
output_df = df_test[['ID', '预测_总价']]  # 假设测试集有 ID 列
output_df.to_csv("C:/Users/HP/Desktop/final/prediction_output.csv", index=False)
